# 13. 딥네트워크, 서로 뭐가 다른 거죠?

### 실습목표
* 딥러닝 네트워크들을 동물원 구경하듯 둘러봅니다. (Model Zoo라고도 하죠)
* 딥러닝이 어떻게 시작된 것인지 알아봅니다.
* ResNet과 VGG는 무엇을 시도했는지 알아봅니다.

### AlexNet
* 5개의 컨볼루션 레이어와 3개의 full-connected 레이어로 구성
* ReLU 활성화 함수와 드롭아웃(Dropout), 그리고 오버래핑 풀링(Overlapping pooling) 등이 적용  
* [AlexNet 논문](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)

![AlexNet](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F99FEB93C5C80B5192E)




### VGG
* 간결한 구조로 ImageNet에서 준우승을 거둔 모델.
* 3*3의 작은 필터를 사용하여 이전의 모델들 보다 깊은 층을 쌓았다.
![vgg](https://d3s0tskafalll9.cloudfront.net/media/images/F-22-6.max-800x600.png)

### Vanising gradient

![vanising_gradient](https://d3s0tskafalll9.cloudfront.net/media/images/F-22-7.max-800x600.png)

* 모델이 깊어지면 모델이 학습하기 위한 gradient값이 매우 작아져서 학습이 느려진다.또한 이 값이 매우 커지기도 하는데 이를 exploding gradients라고 한다.

### ResNet

* 다들 기울기 소실로 깊은 레이어를 쌓지 못할 때 **Skip connection**이라는 구조를 이용해서 100개가 넘는 층을 가지고, 기울기 소실 문제를 어느정도 해결했다.
* **Skip connection**은 이전의 생성된 결과를 몇개의 레이어를 건너뛰고 활성함수 이전에 반영하여 계산하는 방법을 말한다.

![skipConnection](https://d3s0tskafalll9.cloudfront.net/media/original_images/F-22-10.png)

## 13-9. VGG-16

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# CIFAR100 데이터세트
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

In [14]:
print("입력데이터 크기 : ",x_train.shape) # 32*32*3의 rgb이미지가 50000개
print("정답데이터 크기 : ",y_train.shape) # 숫자로 라벨이 나누어져있다.

입력데이터 크기 :  (50000, 32, 32, 3)
정답데이터 크기 :  (50000, 1)


In [16]:
img_input = keras.Input(shape=x_train.shape[1:])
# print(x_train.shape[1:])

x = keras.layers.Conv2D(16, 3, activation='relu')(img_input)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=img_input, outputs=predictions)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 30, 16)        448       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 15, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 32)        4640      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1152)              0   

In [17]:
from keras.engine.training import optimizer
# 모델 학습
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1)

1563/1563 [==============================] - 50s 31ms/step - loss: 3.6187 - accuracy: 0.1542
